# Sea level

In [1]:
using Distributions, Compat, Plots, Random, CSV, DataFrames, GLM
plotly()
upscale = 1
default(line = (:steppre, 0.5, 1, 2), size=(800*upscale,600*upscale))

Wir lesen die Datei ein, entfernen fehlende Werte und erhalten:

In [2]:
data = CSV.File("sea-level.csv") |> DataFrame # Read csv
data[:GMSL] = convert(Array{Float64, 1}, data[:GMSL]) # Convert col to number

data[:Row] = collect(1:size(data)[1]) # Add row number for linear regression
describe(data)

,variable,mean,min,median,max,nunique,nmissing,eltype
,Symbol,Union…,Any,Union…,Any,Union…,Union…,DataType
1,Time,,1993-01-15,,2015-02-15,266,0,Date
2,GMSL,36.0286,-3.5,36.7,79.5,,,Float64
3,Row,133.5,1,133.5,266,,,Int64


In [3]:
first(data, 6)

,Time,GMSL,Row
,Dates…⍰,Float64,Int64
1,1993-01-15,-1.6,1
2,1993-02-15,-3.4,2
3,1993-03-15,5.5,3
4,1993-04-15,0.1,4
5,1993-05-15,5.3,5
6,1993-06-15,0.2,6


Die Entwicklung der Meeresspiegel ist folgendes:

In [4]:
# range = 0:270
plot(data[:Row], data[:GMSL], label = "Entwicklung der Meeresspiegel")

In [5]:
linearmodel = fit(LinearModel, @formula(GMSL ~ Row), data)

┌ Warning: In the future eachcol will have names argument set to false by default
│   caller = evalcontrasts(::DataFrame, ::Dict{Any,Any}) at modelframe.jl:124
└ @ StatsModels /home/meszlenyilorant/.julia/packages/StatsModels/AYB2E/src/modelframe.jl:124


StatsModels.DataFrameRegressionModel{LinearModel{LmResp{Array{Float64,1}},DensePredChol{Float64,LinearAlgebra.Cholesky{Float64,Array{Float64,2}}}},Array{Float64,2}}

Formula: GMSL ~ 1 + Row

Coefficients:
             Estimate  Std.Error  t value Pr(>|t|)
(Intercept)  0.253225   0.415202 0.609883   0.5425
Row           0.26798 0.00269597  99.4001   <1e-99


In [6]:
coef(linearmodel)
error = stderror(linearmodel)[1]

0.415201779211427

In [7]:
f(x) = coef(linearmodel)[2] * x + coef(linearmodel)[1]
plot!(f, 1, size(data)[1], label = "Lineare Schätzung des Modells")

In [8]:
include("../KalmanFilter.jl")
import .KalmanFilter

In [9]:
A = 1 # Zustandsübergangsmatrix
G = 1
Q = 0.5
R = 36 * error

x̂_0 = data[1, :GMSL]
# x̂_0 = f(1)
Σ_0 = -1.6

y = (step) -> data[step, :GMSL] #+ rand(Normal(0, R))

model = KalmanFilter.Kalman(A, Q, G, R, x̂_0, Σ_0)

length = 266
predicted = zeros(length)
fusioned = zeros(length)
confidence = zeros(length)
gains = zeros(length)
for i = 1:length
    y_cur = y(i)
    
    
    KalmanFilter.fusion(model, y_cur)
    fusioned[i] = model.x̂
    
    
    KalmanFilter.predict(model)
    predicted[i] = model.x̂
    
    confidence[i] = model.Σ
    gains[i] = KalmanFilter.kalmanGain(model)
end
plot(xlim = (1, length), ylim = (-3, f(length)))
plot!(predicted, label = "Vorhersage")
plot!(data[:Row], data[:GMSL], label = "Wahrhafte Messung")
plot!(fusioned, label = "Korrigierte Messung")
plot!(f, label = "Lineare Regression")

UndefVarError: UndefVarError: kalmanGain not defined

In [10]:
plot(gains[1:length], label = "Kalman Gewinn", ylim = (0, 1), linewidth = 2)

In [11]:
filteredData = DataFrame(Row = 1:length, GMSL_Filtered = fusioned)
describe(filteredData)

,variable,mean,min,median,max,nunique,nmissing,eltype
,Symbol,Float64,Real,Float64,Real,Nothing,Nothing,DataType
1,Row,133.5,1,133.5,266,,,Int64
2,GMSL_Filtered,-0.00601504,-1.6,0.0,0.0,,,Float64


In [12]:
filteredModel = fit(LinearModel, @formula(GMSL_Filtered ~ Row), filteredData)
g(x) = coef(filteredModel)[2] * x + coef(filteredModel)[1]
plot(xlim = (1, 200))
plot!(filteredData[:GMSL_Filtered], linewidth = 2, label = "Gefilterte Daten", linestyle=:solid)
plot!(f, 1, size(data)[1], label = "Lineare Schätzung <br /> des Modells")
plot!(g, 1, size(data)[1], label = "Lineare Schätzung des <br /> gefilterten Modells")